Tài liệu này mang giấy phép Creative Commons Attribution (CC BY).
(c) Nguyễn Ngọc Sáng, Zhukovsky 7/2020.

[@SangVn](https://github.com/SangVn) [@VnCFD](https://vncfdgroup.wordpress.com/)

*Thực hành CFD với Python!*

**Tài liệu tham khảo**

* 1. V. V. Vlasenko, “Mathematical approach and principles of developing numerical methods for the EWT-TsAGI application package,” Trudy TsAGI, No. 2671 (2007).
* 2. F. Moukalled, L. Mangani, M. Darwish, "The Finite Volume Method in Computational Fluid Dynamics. An Advanced Introduction with OpenFOAM and Matlab"

# Bài 26. Phương pháp tính dòng khuếch tán 

Các khái niệm, công thức cơ bản được sử dụng để tính dòng khuếch tán trong VnCFD_v3.

## 1. Tọa độ cong (curvilinear coordinates)

Để tính được dòng khuếch qua các mặt của ô lưới, chúng ta cần phải xác định được gradient của $(T, u, v)$ tại đó. Ví dụ, ta có công thức xấp xỉ $(T_x)_{_{i+1/2}} = \left(\frac{\partial T}{\partial x}\right)_{_{i+1/2}} = \frac{T_{i+1}-T_{i}}{x_{i+1}-x_{i}}$, tuy nhiên công thức này chỉ có thể sử dụng chính xác trong điều kiện lưới đều Cartesian($x_i=const, y_i=const$). Trường hợp tổng quát, nhiệt độ T được lưu tại các tâm ô lưới nằm trên các đường cong. Do đó để tính gradient ta sẽ chuyển từ không gian vật lý (x, y) sang không gian tọa độ cong $(\xi, \eta)$ gắn với lưới (hình dưới).

<img src="img/mesh.png" width=500>


Khi đó gradient của đại lượng $\phi$ được xác định theo công thức:
\begin{align}
\begin{pmatrix} \phi_x \\ \phi_y \end{pmatrix} =
\begin{pmatrix}
\frac{\partial \xi}{\partial x} & \frac{\partial \xi}{\partial y} \\
\frac{\partial \eta}{\partial x} & \frac{\partial \eta}{\partial y} 
\end{pmatrix}^T
\begin{pmatrix} \phi_{\xi} \\ \phi_{\eta} \end{pmatrix}
\end{align}

với J là ma trận Jacobi:
\begin{equation}
J=
\begin{pmatrix}
\frac{\partial \xi}{\partial x} & \frac{\partial \xi}{\partial y} \\
\frac{\partial \eta}{\partial x} & \frac{\partial \eta}{\partial y} 
\end{pmatrix} =
\begin{pmatrix}
\frac{\partial x}{\partial \xi} & \frac{\partial x}{\partial \eta} \\
\frac{\partial y}{\partial \xi} & \frac{\partial y}{\partial \eta} 
\end{pmatrix}^{-1}
\end{equation}

Đối với vận tốc, ta có:
\begin{align}
\begin{pmatrix} x_t \\ y_t \end{pmatrix} =
\begin{pmatrix}
x_\xi & x_\eta \\
y_\xi & y_\eta
\end{pmatrix}
\begin{pmatrix} \xi_t \\ \eta_t \end{pmatrix}
\end{align}

Hay $V_{\xi, \eta} = J \ V_{x, y}$.


## 2. Ma trận Jacobi 

**Các đạo hàm riêng được tính theo công thức xấp xỉ:** 

$\phi_{\xi}=\frac{\Delta\phi}{\Delta\xi}$ với gia số tọa độ dọc đường lưới $\Delta^L \xi=\sqrt{(\Delta^L x)^2 + (\Delta^L y)^2}, \Delta^R \xi=\sqrt{(\Delta^R x)^2 + (\Delta^R y)^2}, \Delta \xi = \Delta^L \xi + \Delta^R \xi $. 

Xét cụ thể ô lưới (i,j) và mặt (i+1/2,j) như trên hình.
<img src="img/cell.png" width=400>

**Jacobi tại tâm ô lưới (i,j):**
$$\frac{\partial x}{\partial \xi}=\frac{x_{i+1/2,j}-x_{i-1/2,j}}{\sqrt{(x_{i+1/2,j}-x_{i-1/2,j})^2+(y_{i+1/2,j}-y_{i-1/2,j})^2}}$$
$$\frac{\partial y}{\partial \xi}=\frac{y_{i+1/2,j}-y_{i-1/2,j}}{\sqrt{(x_{i+1/2,j}-x_{i-1/2,j})^2+(y_{i+1/2,j}-y_{i-1/2,j})^2}}$$
$$\frac{\partial x}{\partial \eta}=\frac{x_{i,j+1/2}-x_{i,j-1/2}}{\sqrt{(x_{i,j+1/2}-x_{i,j-1/2})^2+(y_{i,j+1/2}-y_{i,j-1/2})^2}}$$
$$\frac{\partial y}{\partial \eta}=\frac{y_{i,j+1/2}-y_{i,j-1/2}}{\sqrt{(x_{i,j+1/2}-x_{i,j-1/2})^2+(y_{i,j+1/2}-y_{i,j-1/2})^2}}$$

Đây là các công thức xấp xỉ sai phân trung tâm (central difference).

**Jacobi tại tâm mặt (i+1/2,j):**

Theo phương tiếp tuyến với bề mặt:
$$\frac{\partial x}{\partial \eta}=\frac{x_{i+1/2,j+1/2}-x_{i+1/2,j-1/2}}{\sqrt{(x_{i+1/2,j+1/2}-x_{i+1/2,j-1/2})^2+(y_{i+1/2,j+1/2}-y_{i+1/2,j-1/2})^2}}$$
$$\frac{\partial y}{\partial \eta}=\frac{y_{i+1/2,j+1/2}-y_{i+1/2,j-1/2}}{\sqrt{(x_{i+1/2,j+1/2}-x_{i+1/2,j-1/2})^2+(y_{i+1/2,j+1/2}-y_{i+1/2,j-1/2})^2}}$$

Riêng đối với $x_\xi, y_\xi$, trong trường hợp tổng quát điểm (i+1/2,j) có thể không nằm trên đoạn nối tâm hai ô lưới. Do đó để xác định chúng ta sử dụng hàm nội suy:
$$x_\xi = interpolation(x_\xi^L, x_\xi^R, \Delta \xi^L, \Delta \xi^R)=\frac{\Delta \xi^R x_\xi^L + \Delta \xi^L x_\xi^R}{\Delta \xi^L+\Delta \xi^R}$$
$$x_\xi^L=\frac{x_{i+1/2,j}-x_{i,j}}{\Delta \xi^L}, \Delta \xi^L = \sqrt{(x_{i+1/2,j}-x_{i,j})^2+(y_{i+1/2,j}-y_{i,j})^2}$$
$$x_\xi^R=\frac{x_{i+1,j}-x_{i+1/2,j}}{\Delta \xi^R}, \Delta \xi^R = \sqrt{(x_{i+1,j}-x_{i+1/2,j})^2+(y_{i+1,j}-y_{i+1/2,j})^2}$$
Tương tự cho $y_\xi$.

Đối với các mặt nằm trên biên của vùng tính toán, phụ thuộc theo vị trí của chúng ta lấy: $x_\xi=x_\xi^L$ hoặc $x_\xi=x_\xi^R$.

## 3. Gradient 
Gradient $(\phi_x, \phi_y)$ với $\phi$ là các đại lượng $(T, u, v)$ được tính theo công thức: $(\phi_x, \phi_y)^T=J(\phi_\xi, \phi_\eta)^T$.

**Gradient tại tâm ô lưới (i,j):**

Trong trường hợp lưới đều, ta có thể sử dụng công thức xấp xỉ sai phân trung tâm:
$$\phi_\xi = \frac{\phi_{i+1,j}-\phi_{i-1,j}}{\Delta \xi}, \Delta \xi = \sqrt{(x_{i+1,j}-x_{i-1,j})^2+(y_{i+1,j}-y_{i-1,j})^2}$$
Trường hợp tổng quát, ta sử dụng hàm nội suy:
$$\phi_\xi = interpolation(\phi_\xi^L, \phi_\xi^R, \Delta\xi^L, \Delta\xi^R)$$ với 

$$\phi_\xi^L=\frac{\phi_{i,j}-\phi_{i-1,j}}{\Delta \xi^L}, \Delta \xi^L = \sqrt{(x_{i,j}-x_{i-1,j})^2+(y_{i,j}-y_{i-1,j})^2}$$
$$\phi_\xi^R=\frac{\phi_{i+1,j}-\phi_{i,j}}{\Delta \xi^R}, \Delta \xi^R = \sqrt{(x_{i+1,j}-x_{i,j})^2+(y_{i+1,j}-y_{i,j})^2}$$
Tương tự cho $\phi_\eta$.

**Gradient tại tâm mặt (i+1/2,j):**

Theo phương tiếp tuyến với mặt, gradient được tính thông qua hàm nội suy hai giá trị tại hai ô lưới hai bên: $$(\phi_\eta)_{i+1/2,j} = interpolation((\phi_\eta)_{i,j}, (\phi_\eta)_{i+1,j}, (\Delta\xi)_{i,j}, (\Delta\xi)_{i+1,j})$$
Theo phương còn lại, nếu lưới đều thì tâm mặt $C$ nằm giữa đường nối hai tâm ô lưới $(i, j)$ và $(i+1, j)$ - tại điểm có $\xi_C=0.5(\xi_{i, j} + \xi_{i+1,j})$, khi đó, ta có công thức xấp xỉ bậc 2: 
$$(\phi_\xi)_{i+1/2,j} = (\phi_\xi)_C \approx \frac{\Delta \phi}{\Delta \xi} = \frac{\phi_{i+1, j} - \phi_{i, j}}{\xi_{i+1,j}-\xi_{i,j}}$$
Trường hợp tổng quát, sử dụng công thức khai triển:
$$(\phi_\xi)_{i+1/2,j}=(\phi_\xi)_C+\left(\frac{\partial \phi_\xi}{\partial \xi}\right)_C (\xi_{i+1/2,j}-\xi_C) + O((\Delta \xi)^2)\\
=(\phi_\xi)_C+\left(\frac{\partial \phi_\xi}{\partial \xi}\right)_C (\xi_{i+1/2,j}-0.5(\xi_{i,j}+\xi_{i+1,j})) + O((\Delta \xi)^2)\\
=(\phi_\xi)_C+\left(\frac{\partial \phi_\xi}{\partial \xi}\right)_C 0.5(\Delta \xi^L-\Delta \xi^R) + O((\Delta \xi)^2)\\
\approx \frac{\phi_{i+1, j} - \phi_{i, j}}{\Delta \xi^L + \Delta \xi^R}
+ \frac{(\phi_\xi)_{i+1,j}-(\phi_\xi)_{i,j}}{\Delta \xi^L + \Delta \xi^R}0.5(\Delta \xi^L-\Delta \xi^R)$$

## 4. Độ nhớt động lực học, mật độ dòng nhiệt

Để tính độ nhớt động lực học hay mật độ dòng nhiệt tại các mặt chúng ta phải sử dụng giá trị nhiệt độ T nào cho phù hợp? Ở đây ta sẽ xác định nhiệt độ T trên các mặt ô lưới thông qua hàm nội suy giá trị $(\rho, p)$ ở hai ô lưới hai bên. Trong VnCFD, tại tâm các lưới ta chỉ lưu các thông số $P=(\rho, u, v, p)$ mà không có T. Tại các mặt ô lưới $P$ xác định theo công thức:
$$P_{i+1/2,j} = interpolation(P_{i,j}, P_{i+1,j}, \Delta \xi^L, \Delta \xi^R)$$

Khi đó, tại các mặt:
$$\mu = \mu(T)$$
$$gradT = grad(p/(R\rho)) = grad p/(R\rho) - p\ grad\rho/(R\rho^2)$$

## 5. GhostCell

Để thuận tiện cho việc tính toán ở trên biên hay ở các ô lưới kề biên, người ta thường sử dụng thêm một "ô lưới ma" bên ngoài các mặt trên biên vùng tính toán gọi là "Ghost Cell". Trong VnCFD_v3, GhostCell có kích thước bằng không, và chứa các thông số dòng chảy tại mặt trên biên kề nó. 

## 6. Tổng kết 

Như vậy, để tính dòng khuếch tán, chúng ta cần bổ sung vào solver Euler các bước sau:

1. Tính ma trận Jacobi tại các mặt ô lưới
2. Tính Gradient theo tọa độ $\xi, \eta$ dọc đường lưới tại tâm các ô lưới và tại các mặt ô lưới
3. Tính Gradient theo tọa độ $x,y$ tại các mặt ô lưới
4. Tính giá trị các thông số $P=(\rho, u, v, p)$ tại các mặt ô lưới dùng hàm nội suy
5. Tính độ nhớt động lực học, tesor độ nhớt, mật độ dòng nhiệt tại các mặt
6. Tính dòng khuếch tán và bổ sung vào dòng qua mặt.

Ngoài các công thức tính gradient ở trên dành cho lưới có cấu trúc, có thể sử dụng các công thức cho lưới không cấu trúc như trong tài liệu tham khảo thứ 2 về OpenFoam.

# [Bài 27. Điều kiện biên](Bai_27.ipynb)